In [10]:
game = {   "variant" : "NT",
    "game_id" : 808894969,
    "hand_nb" : 0,
 
    "small_blind" : 0.50,
    "big_blind" : 1,
    "min_bet" : 0,
 
    "players" : ["Gargantuatua", "LAGrinder", "IlxxxlI", "MrPink", "MrBrown", "Pluribus"],
    "starting_stacks" : [10000, 10000, 10000, 10000, 10000, 10000],
    "players_seats" : [1, 2, 3, 4, 5, 6],
 
    "button_seat" : 1,
    "player_small_blind" : "Gargantuatua",
    "player_big_blind" : "LAGrinder",
 
    "player" : "IlxxxlI",
    "cards_player" : ["10h", "10c"],
 
    "dealed_cards" : {
                    "flop": ["Qs", "9c", "4s"],
                    "turn": ["As"],
                    "river": ["8d"]
                   },
 
 
    "actions" : {"pre-flop" : {"players": ["IlxxxlI", "MrPink", "MrBrown", "Pluribus", "Gargantuatua", "LAGrinder"],
                            "actions": ["cbr", "f", "cbr", "f", "cc", "cc"],
                            "value": [None, None, 265, None, 1, 1]},
                "post-flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post-turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post-river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 
 
    "winners" : ["IlxxxlI"],
    "finishing_stacks": [10112.5, 9775.0, 10000.0, 10000.0, 10112.5, 10000.0],
    "card_shown_by_players" : [None, None, None, None, None, None]
}
# player = "IlxxxlI"
# player_index = game["actions"]["pre-flop"]["players"].index(player)
# print(player_index)
# print('cbr' in game["actions"]["pre-flop"]["actions"][player_index])

0
True


In [2]:
def compute_pfr(games, player="IlxxxlI"):
   """entrée: liste de formated game. 
   Calcule de la pre flop raise pour un set de partie"""
   total_hands = 0
   preflop_raises = 0
   for game in games:
       
        if player not in game["players"]:
            return 'mauvais nom de joueurs'

        player_index = game["actions"]["pre-flop"]["players"].index(player)
        preflop_actions = game["actions"]["pre-flop"]["actions"][player_index]

        if "cbr" in preflop_actions:
            preflop_raises += 1
        total_hands +=1
   # Calculate PFR
   if total_hands == 0:
       return 0
   else:
       pfr = (preflop_raises / total_hands)*100
       print("compute pfr, Ideally, it should be 2 or 3%: " + pfr)
       return pfr

Action possible pour le llm cyber: bets, calls, raises , allin, checks, caps

In [3]:
def is_legal_action(output_llm):
    """
    Entrée: 1 Sortie brut du llm. 
    Vérifie si le coup proposé par le llm fait partie du set de coup possible
    """
    valid_actions = {"bets", "calls", "raises", "allin", "checks", "caps", "folds"}
    if any(action in output_llm.split() for action in valid_actions):
        return True
    return False

def legal_actions(outputs_llm):
    """ Entrée: liste de sortie llm brute
    Compte le nombre de coups non autorisé 
    """
    count=0
    for output in outputs_llm: 
        if is_legal_action(output): 
            count+=1
    return count

# test_output="raises (2.3). <\s>"
# print('raises' in test_output.split())


In [44]:
import re
import pandas as pd

def parse_action(action): 
    """parser pour ne pas prendre en compte les montants joués"""
    action = action.strip()
    match= re.match(r'(\w+)', action)
    return match.group(1) if match else None


legal_moves = ['bets', 'calls', 'raises', 'allin', 'checks', 'folds', 'caps']
def compare_to_test(waited_outputs, llm_outputs):
    """ 
    Entrée: output test set brute, output llm
    à partir d'une list des résultats attendue vs résultat prédit créer une matrice de confusion
    """
    waited_normalized = [parse_action(action) for action in waited_outputs]
    llm_normalized = [parse_action(action) for action in llm_outputs]
    confusion_matrix = pd.DataFrame(0, index=legal_moves, columns=legal_moves)
 
    for waited, llm in zip(waited_normalized, llm_normalized):
        if waited in legal_moves and llm in legal_moves:
            confusion_matrix.at[waited, llm] += 1
    return confusion_matrix

def find_misclassifications(waited_outputs, llm_outputs, true_action="raises", predicted_action="folds"):     
    indices = []   
    for i, (waited, llm) in enumerate(zip(waited_outputs, llm_outputs)):         
        waited_norm = parse_action(waited)        
        llm_norm = parse_action(llm)        
        if waited_norm == true_action and llm_norm == predicted_action: 
            indices.append(i)
    return indices
 
# Test

# waited_outputs = [
#     "raises (3)",
#     "calls (2)",
#     "checks",
#     "bets (5)",
#     "folds"
# ]
 
# llm_outputs = [
#     "raises (5.4) <s>",
#     "calls (2)",
#     " checks", 
#     "bets (10)",
#     "folds"
# ]
# confusion_matrix = compare_to_test(waited_outputs, llm_outputs)
# print(confusion_matrix)


        bets  calls  raises  allin  checks  folds  caps
bets       1      0       0      0       0      0     0
calls      0      1       0      0       0      0     0
raises     0      0       1      0       0      0     0
allin      0      0       0      0       0      0     0
checks     0      0       0      0       1      0     0
folds      0      0       0      0       0      1     0
caps       0      0       0      0       0      0     0


In [ ]:
def create_game(carte1, carte2): 
    return game
def inference(game): 
    "api mistral "
    return action

tableau= dataframe(cartes1, carte2) #tableau avec à chaque case associé les actions réalisés
#ligne = carte 1; colonne = carte 2 
# contenue {fold:, raise: , checks: }

for carte1 ...  
    for carte 2... 
        game=create_game(carte1, carte2)
        action = inference(game)
        tableau(carte1, carte2)[action] += 1

